<a href="https://colab.research.google.com/github/kw908/RL_doorkick/blob/main/Suttonbookexercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 3.18 \\
Relationship between $V^\pi$ and $q^\pi$:
$$
V^\pi(s) = \sum_{a \in A} \pi(a|s)q^\pi(s,a)
$$

In [ ]:
## Exercise 3.14

import numpy as np

neighbors = [(0,1.0,2.3),
                   (0,1.0,0.4),
                   (0,1.0,-0.4),
                   (0,1.0,0.7)] # reward, probability, value(only accurate to one decimal place)
policy = np.array([0.25,0.25,0.25,0.25])
rewards = np.array([0,0,0,0])
middle_value = 0.7
gamma = 0.9
v_pi_middle = 0
state = [0,1]

for pa,v in zip(policy,neighbors):
       v_pi_middle += pa * v[1] * (v[0] + gamma*v[2])

print(v_pi_middle) #should evaluate to near 0.7


In [1]:
## Example 4.1
import math
from Environments import gridworld_v0 as gw

THETA = 0.2 #stop condition
GAMMA = 1.0 #non-discounted
episodes = 100 # number of episodes
k_terminal = 0
env = gw.environment(sidelength=4,episodes=episodes) 

def policy_eval():
    for k in range(episodes):
        delta=0
        for s in env.states:
            v=s.value[k]
            for a in env.actions:
                next_state_alias, next_state_value, reward = env.step(s,a,k)
                s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value)) # calculate q
                s.value[k+1] += s.policy[a][0] * s.policy[a][2] # calculate v
            delta = max(delta, abs(v-s.value[k+1]))
        if delta<THETA: 
            print("Done evaluation~~~")
            k_terminal = k
            break

    return k_terminal

def policy_imp(k_terminal):
    for s in env.states:
        a_opt = []
        for a in env.actions:
            next_state_alias, next_state_value, reward = env.step(s,a,k_terminal) # take one more step won't hurt I guess??
            s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value))
        # update policy
        for a in env.actions:
            if s.policy[a][2] == max([i[2] for i in s.policy.values()]):
                a_opt.append(a) #find optimal action(s)
        for a in a_opt:
            s.policy[a][2] = 1/len(a_opt) #equally distribute probabilities among optimal actions

delta=0
v=0

while(True):
    policy_eval()
    policy_imp()


    break

# while(True):
#     policy_eval()
#     policy_imp()
#     break
        
for k in range(episodes):
    print("k is ",k)
    for s in env.states:
        print(math.ceil(s.value[k+1]*10)/10)
    print("hahaha")
# """policy improvement"""

# policy_stable = True
# k=0
# for s in env.states:
#     old_action = env.policy[k]

#     env.policy[k+1] = 


       



Done evaluation~~~
k is  0
0.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
0.0
hahaha
k is  1
0.0
-1.7
-2.0
-2.0
-1.7
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-1.7
-2.0
-2.0
-1.7
0.0
hahaha
k is  2
0.0
-2.4
-2.9
-2.9
-2.5
-2.8
-3.0
-2.9
-2.9
-3.0
-2.8
-2.5
-2.9
-2.9
-2.4
0.0
hahaha
k is  3
0.0
-3.0
-3.8
-3.8
-3.1
-3.7
-3.9
-3.8
-3.8
-3.9
-3.7
-3.1
-3.8
-3.8
-3.0
0.0
hahaha
k is  4
0.0
-3.6
-4.6
-4.6
-3.8
-4.5
-4.7
-4.6
-4.6
-4.7
-4.5
-3.8
-4.6
-4.6
-3.6
0.0
hahaha
k is  5
0.0
-4.2
-5.4
-5.4
-4.4
-5.2
-5.5
-5.5
-5.5
-5.5
-5.2
-4.4
-5.4
-5.4
-4.2
0.0
hahaha
k is  6
0.0
-4.7
-6.1
-6.2
-5.0
-5.9
-6.3
-6.2
-6.2
-6.3
-5.9
-5.0
-6.2
-6.1
-4.7
0.0
hahaha
k is  7
0.0
-5.2
-6.8
-6.9
-5.6
-6.6
-7.0
-6.9
-6.9
-7.0
-6.6
-5.6
-6.9
-6.8
-5.2
0.0
hahaha
k is  8
0.0
-5.6
-7.5
-7.5
-6.1
-7.2
-7.7
-7.6
-7.6
-7.7
-7.2
-6.1
-7.5
-7.5
-5.6
0.0
hahaha
k is  9
0.0
-6.1
-8.1
-8.2
-6.6
-7.8
-8.4
-8.2
-8.2
-8.4
-7.8
-6.6
-8.2
-8.1
-6.1
0.0
hahaha
k is  10
0.0
-6.5
-8.7
-8.8
-7.0
-8.3
-9.0
-8.8
-8.